In [1]:
import pandas as pd
import geopandas as gpd

#load the data and show that we have correctly parsed the CSV
with open("Resources_Organized/Bestand3.csv") as file:
    data = pd.read_csv(file,sep=";")
data.head

<bound method NDFrame.head of       id                  x                 y               BHD
0      1  -68,2167006832299  61,3956239130436  56,1497142857143
1      2  -42,1984159281437  22,2948881437126  15,1288982035928
2      3  -35,6605485714286  48,7894283809524  34,5655619047619
3      4         -63,811228  60,6850993333333           32,0572
4      5  -12,3950435835351   77,242318062954  38,3888232445521
..   ...                ...               ...               ...
585  586  -36,9743130434783  38,8572591304348  28,9052173913044
586  587   -52,159067823765  38,5023822162884  40,0141842456609
587  588  -25,4799277966102  77,1735644067797  42,7526101694916
588  589  -8,60909308411215   75,078196728972  20,3729345794393
589  590  -26,3242885714286  59,0683546666667  46,8279238095239

[590 rows x 4 columns]>

#### Bird Eye Problem Approach

Set-up the map
1. Create a spatial map from the largest coordinates of each dataset
2. Add the trees to the map, classifying them wrt BHD (and possibly other factors)
3. Somehow designate the road area? Or should this happen manually?